In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
np.random.seed(1000)
import pathlib

In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
print(data_dir)
data_dir = pathlib.Path(data_dir).with_suffix('')
print(data_dir)

228813984/228813984 [==============================] - 1s 0us/step
/root/.keras/datasets/flower_photos.tar
/root/.keras/datasets/flower_photos


In [3]:
batch_size=128
img_height=227
img_width=227

In [4]:
train_ds1=tf.keras.utils.image_dataset_from_directory(
    data_dir,validation_split=0.2,
    subset="training",
    seed=123, #the same set of images always go into the training, test, validation set.
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [5]:
class_names = train_ds1.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [6]:
val_ds1=tf.keras.utils.image_dataset_from_directory(
    data_dir,validation_split=0.2,
    subset="validation",
    seed=123, #the same set of images always go into the training, test, validation set.
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [7]:
num_classes=len(class_names)

In [8]:
model0=Sequential()
#layer 1
model0.add(Conv2D(filters=96,input_shape=(227,227,3),kernel_size=(11,11),strides=(4,4),padding='valid'))
model0.add(Activation("relu"))
model0.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))

#layer2
model0.add(Conv2D(filters=96,input_shape=(227,227,3),kernel_size=(5,5),strides=(1,1),padding='valid'))
model0.add(Activation("relu"))
model0.add(MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))

#layer3
model0.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid'))
model0.add(Activation("relu"))


#layer4
model0.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid'))
model0.add(Activation("relu"))

#layer5
model0.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='valid'))
model0.add(Activation("relu"))

#final pooling
model0.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

#fully connected
model0.add(Flatten())

#1st FC
model0.add(Dense(4096, input_shape=(227*227*3,)))
model0.add(Activation('relu'))

#dropout to prevent overfitting
model0.add(Dropout(0.4))

#2nd FC
model0.add(Dense(4096))
model0.add(Activation('relu'))
#dropout
model0.add(Dropout(0.4))

#3rd FC
model0.add(Dense(1000))
model0.add(Activation('relu'))
#dropout
model0.add(Dropout(0.4))

#output layer
model0.add(Dense(num_classes))
model0.add(Activation('relu'))

In [9]:
#model compile
model0.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #categorical clasification
              metrics=['accuracy'])

In [10]:
model0.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 activation (Activation)     (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 96)        230496    
                                                                 
 activation_1 (Activation)   (None, 23, 23, 96)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 96)       0         
 2D)                                                    

In [11]:
#model fit
epochs=15
history0 = model0.fit(train_ds1, validation_data=val_ds1, epochs=epochs)

Epoch 1/15
 3/23 [==>...........................] - ETA: 2:57 - loss: 6.3825 - accuracy: 0.1510

KeyboardInterrupt: ignored

In [ ]:
acc = history0.history['accuracy']
val_acc = history0.history['val_accuracy']
loss = history0.history['loss']
val_loss = history0.history['val_loss']
epochs_range=range(epochs)
plt.figure(figsize=(8,8))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()